

```
# This is formatted as code
```

# Capstone Project: NLP - Safebot - FastText

In [1]:
# Importing the library
#import plotly
#print(plotly.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objects as go
#import plotly.io as pio
#pio.renderers.default = 'colab'
from sklearn.impute import SimpleImputer
# Basic packages
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, gc
from scipy import stats; from scipy.stats import zscore, norm, randint
import matplotlib.style as style; style.use('fivethirtyeight')
# Models
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, learning_curve
#from wordcloud import WordCloud, STOPWORDS

# Display settings
pd.options.display.max_rows = 400
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format

random_state = 42
np.random.seed(random_state)

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('industrial_safety_and_health_database_with_accidents_description.csv')
print(df.shape)
df.head()

(425, 11)


,Unnamed: 0,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


In [3]:
ds = df.copy()
ds.drop('Unnamed: 0',1, inplace = True)

In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Data                      425 non-null    object
 1   Countries                 425 non-null    object
 2   Local                     425 non-null    object
 3   Industry Sector           425 non-null    object
 4   Accident Level            425 non-null    object
 5   Potential Accident Level  425 non-null    object
 6   Genre                     425 non-null    object
 7   Employee or Third Party   425 non-null    object
 8   Critical Risk             425 non-null    object
 9   Description               425 non-null    object
dtypes: object(10)
memory usage: 33.3+ KB


In [5]:
# Displaying the columns
ds.columns

Index(['Data', 'Countries', 'Local', 'Industry Sector', 'Accident Level',
       'Potential Accident Level', 'Genre', 'Employee or Third Party',
       'Critical Risk', 'Description'],
      dtype='object')

In [6]:
ds.rename(columns= {'Data':'Date', 'Countries':'Country', 'Genre':'Gender',
                    'Employee or Third Party':'Employee type'}, inplace =True)
ds.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description
0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...
3,2016-01-08 00:00:00,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...
4,2016-01-10 00:00:00,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...


## Data Preparation for Time Series Analysis

In [7]:
ds['Date'] = pd.to_datetime(ds['Date'])
ds['Year'] = ds['Date'].apply(lambda x: x.year)
ds['Month'] = ds['Date'].apply(lambda x: x.month)
ds['Day'] = ds['Date'].apply(lambda x: x.day)
ds['Weekday'] = ds['Date'].apply(lambda x: x.day_name())
ds.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description,Year,Month,Day,Weekday
0,2016-01-01,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday
1,2016-01-02,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday
2,2016-01-06,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday
3,2016-01-08,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday
4,2016-01-10,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday


In [8]:
# Defining a function to creat new feature called Quater
def month_quater_conversion(x):
  if x in [1, 2, 3]:
    season = 'First'
  elif x in [4, 5, 6]:
    season = 'Second'
  elif x in [7, 8, 9]:
    season = 'Third'
  elif x in [10, 11, 12]:
    season = 'Fourth'
  return season

ds['Quater'] = ds['Month'].apply(month_quater_conversion)
ds.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description,Year,Month,Day,Weekday,Quater
0,2016-01-01,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,First
1,2016-01-02,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,First
2,2016-01-06,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,First
3,2016-01-08,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,First
4,2016-01-10,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,First


In [9]:
ds['Accident Level'].unique()

array(['I', 'IV', 'III', 'II', 'V'], dtype=object)

In [10]:
ds['Potential Accident Level'].unique()

array(['IV', 'III', 'I', 'II', 'V', 'VI'], dtype=object)

In [11]:
# Defining the level as per Accident Level
def Accident_level_conversion(x):
  if x in ['I','II']:
    level = 'Low'
  elif x in ['III']:
    level = 'Medium'
  elif x in ['IV', 'V', 'VI']:
    level = 'High'
  return level

In [12]:
ds['Accident Severity'] = ds['Accident Level'].apply(Accident_level_conversion)

In [13]:
ds['Potential Severity'] = ds['Potential Accident Level'].apply(Accident_level_conversion)

In [14]:
ds.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description,Year,Month,Day,Weekday,Quater,Accident Severity,Potential Severity
0,2016-01-01,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,First,Low,High
1,2016-01-02,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,First,Low,High
2,2016-01-06,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,First,Low,Medium
3,2016-01-08,Country_01,Local_04,Mining,I,I,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,First,Low,Low
4,2016-01-10,Country_01,Local_04,Mining,IV,IV,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,First,High,High


## Replacing categorical values

In [15]:
replace_value = {'Local_01':1, 'Local_02':2, 'Local_03':3, 'Local_04':4, 'Local_05':5, 'Local_06':6, 'Local_07':7, 'Local_08':8, 
                 'Local_10':10, 'Local_09':9, 'Local_11':11, 'Local_12':12}
ds['Local'] = ds['Local'].map(replace_value)

replace_value = {'I':1, 'IV':4, 'III':3, 'II':2, 'V':5}
ds['Accident Level'] = ds['Accident Level'].map(replace_value)

replace_value = {'IV':4, 'III':3, 'I':1, 'II':2, 'V':5, 'VI':6}
ds['Potential Accident Level'] = ds['Potential Accident Level'].map(replace_value)

del replace_value

ds.head()

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description,Year,Month,Day,Weekday,Quater,Accident Severity,Potential Severity
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,First,Low,High
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,First,Low,High
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,First,Low,Medium
3,2016-01-08,Country_01,4,Mining,1,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,First,Low,Low
4,2016-01-10,Country_01,4,Mining,4,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,First,High,High


# Exploratory Data Analysis

In [16]:
# Removong stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
ds['clean_Description'] = ds['Description'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Design, train and test with Fast Text classifiers

# Installing and using Fastext now for Classification

In [17]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [18]:
import fasttext
import csv


In [19]:
ds.head(10)

,Date,Country,Local,Industry Sector,Accident Level,Potential Accident Level,Gender,Employee type,Critical Risk,Description,Year,Month,Day,Weekday,Quater,Accident Severity,Potential Severity,clean_Description
0,2016-01-01,Country_01,1,Mining,1,4,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,1,Friday,First,Low,High,"While removing drill rod Jumbo 08 maintenance,..."
1,2016-01-02,Country_02,2,Mining,1,4,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,1,2,Saturday,First,Low,High,"During activation sodium sulphide pump, piping..."
2,2016-01-06,Country_01,3,Mining,1,3,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...,2016,1,6,Wednesday,First,Low,Medium,In sub-station MILPO located level +170 collab...
3,2016-01-08,Country_01,4,Mining,1,1,Male,Third Party,Others,Being 9:45 am. approximately in the Nv. 1880 C...,2016,1,8,Friday,First,Low,Low,Being 9:45 am. approximately Nv. 1880 CX-695 O...
4,2016-01-10,Country_01,4,Mining,4,4,Male,Third Party,Others,Approximately at 11:45 a.m. in circumstances t...,2016,1,10,Sunday,First,High,High,Approximately 11:45 a.m. circumstances mechani...
5,2016-01-12,Country_02,5,Metals,1,3,Male,Third Party (Remote),Pressurized Systems,During the unloading operation of the ustulado...,2016,1,12,Tuesday,First,Low,Medium,During unloading operation ustulado Bag need u...
6,2016-01-16,Country_02,5,Metals,1,3,Male,Employee,Fall prevention (same level),The collaborator reports that he was on street...,2016,1,16,Saturday,First,Low,Medium,The collaborator reports street 09 holding lef...
7,2016-01-17,Country_01,4,Mining,1,3,Male,Third Party,Pressed,"At approximately 04:50 p.m., when the mechanic...",2016,1,17,Sunday,First,Low,Medium,"At approximately 04:50 p.m., mechanic technici..."
8,2016-01-19,Country_02,2,Mining,1,4,Male,Third Party (Remote),Others,Employee was sitting in the resting area at le...,2016,1,19,Tuesday,First,Low,High,Employee sitting resting area level 326 (raise...
9,2016-01-26,Country_01,6,Metals,1,2,Male,Third Party,Chemical substances,At the moment the forklift operator went to ma...,2016,1,26,Tuesday,First,Low,Low,At moment forklift operator went manipulate bi...


In [20]:

ft_df_Potential = pd.DataFrame(columns=['fasttext_data'])
ft_df_Accident = pd.DataFrame(columns=['fasttext_data'])

ft_df_Potential['fasttext_data'] ='__label__' + ds['Potential Accident Level'].astype(str) + " "+ds['clean_Description']
ft_df_Accident['fasttext_data'] ='__label__' + ds['Accident Level'].astype(str) + " "+ds['clean_Description']


ft_df_Potential



,fasttext_data
0,__label__4 While removing drill rod Jumbo 08 m...
1,__label__4 During activation sodium sulphide p...
2,__label__3 In sub-station MILPO located level ...
3,__label__1 Being 9:45 am. approximately Nv. 18...
4,__label__4 Approximately 11:45 a.m. circumstan...
...,...
420,__label__3 Being approximately 5:00 a.m. appro...
421,__label__2 The collaborator moved infrastructu...
422,__label__2 During environmental monitoring act...
423,__label__2 The Employee performed activity str...


In [21]:
ft_df_Accident

,fasttext_data
0,__label__1 While removing drill rod Jumbo 08 m...
1,__label__1 During activation sodium sulphide p...
2,__label__1 In sub-station MILPO located level ...
3,__label__1 Being 9:45 am. approximately Nv. 18...
4,__label__4 Approximately 11:45 a.m. circumstan...
...,...
420,__label__1 Being approximately 5:00 a.m. appro...
421,__label__1 The collaborator moved infrastructu...
422,__label__1 During environmental monitoring act...
423,__label__1 The Employee performed activity str...


In [22]:
#ft_df = ft_df[['poten_acc_level','description']]

In [23]:
ft_df_Potential.tail(10)

,fasttext_data
415,__label__2 Employee reports removing zinc shee...
416,__label__4 During execution task assembling bo...
417,__label__4 The technician returning activities...
418,"__label__1 During field activity AMG project, ..."
419,__label__3 After maintenance Boltec 03 level 2...
420,__label__3 Being approximately 5:00 a.m. appro...
421,__label__2 The collaborator moved infrastructu...
422,__label__2 During environmental monitoring act...
423,__label__2 The Employee performed activity str...
424,"__label__2 At 10:00 a.m., assistant cleaned fl..."


In [24]:
ft_df_Accident.tail(10)

,fasttext_data
415,__label__1 Employee reports removing zinc shee...
416,__label__3 During execution task assembling bo...
417,__label__1 The technician returning activities...
418,"__label__1 During field activity AMG project, ..."
419,__label__1 After maintenance Boltec 03 level 2...
420,__label__1 Being approximately 5:00 a.m. appro...
421,__label__1 The collaborator moved infrastructu...
422,__label__1 During environmental monitoring act...
423,__label__1 The Employee performed activity str...
424,"__label__1 At 10:00 a.m., assistant cleaned fl..."


In [25]:

%cd '/content/gdrive/My Drive/NLP_Capstone_Project/'


ft_df_Potential.to_csv(r'fasttext_industrial_Potential.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

ft_df_Accident.to_csv(r'fasttext_industrial_Accident.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")





[WinError 3] The system cannot find the path specified: "'/content/gdrive/My Drive/NLP_Capstone_Project/'"
C:\Users\reach\OneDrive\Great Learning\rasa2\Fasttext


In [26]:
!head fasttext_industrial_Potential.txt

__label__4  While  removing  drill  rod  Jumbo  08  maintenance,  supervisor  proceeds  loosen  support  intermediate  centralizer  facilitate  removal,  seeing  mechanic  supports  one  end  drill  equipment  pull  hands  bar  accelerate  removal  this,  moment  bar  slides  point  support  tightens  fingers  mechanic  drilling  bar  beam  jumbo.
__label__4  During  activation  sodium  sulphide  pump,  piping  uncoupled  sulfide  solution  designed  area  reach  maid.  Immediately  made  use  emergency  shower  directed  ambulatory  doctor  later  hospital.  Note:  sulphide  solution  =  48  grams  /  liter.
__label__3  In  sub-station  MILPO  located  level  +170  collaborator  excavation  work  pick  (hand  tool),  hitting  rock  flat  part  beak,  bounces  hitting  steel  tip  safety  shoe  metatarsal  area  â€‹â€‹the  left  foot  collaborator  causing  injury.
__label__1  Being  9:45  am.  approximately  Nv.  1880  CX-695  OB7,  personnel  begins  task  unlocking  Soquet  bolts  B

In [27]:
!head fasttext_industrial_Accident.txt

__label__1  While  removing  drill  rod  Jumbo  08  maintenance,  supervisor  proceeds  loosen  support  intermediate  centralizer  facilitate  removal,  seeing  mechanic  supports  one  end  drill  equipment  pull  hands  bar  accelerate  removal  this,  moment  bar  slides  point  support  tightens  fingers  mechanic  drilling  bar  beam  jumbo.
__label__1  During  activation  sodium  sulphide  pump,  piping  uncoupled  sulfide  solution  designed  area  reach  maid.  Immediately  made  use  emergency  shower  directed  ambulatory  doctor  later  hospital.  Note:  sulphide  solution  =  48  grams  /  liter.
__label__1  In  sub-station  MILPO  located  level  +170  collaborator  excavation  work  pick  (hand  tool),  hitting  rock  flat  part  beak,  bounces  hitting  steel  tip  safety  shoe  metatarsal  area  â€‹â€‹the  left  foot  collaborator  causing  injury.
__label__1  Being  9:45  am.  approximately  Nv.  1880  CX-695  OB7,  personnel  begins  task  unlocking  Soquet  bolts  B

In [28]:
!wc fasttext_industrial_Potential.txt


   425  16137 131453 fasttext_industrial_Potential.txt


In [29]:
!head -n 340 fasttext_industrial_Potential.txt > ft_ind_Potential.train
!tail -n 85 fasttext_industrial_Potential.txt > ft_ind_Potential.valid

[WinError 3] The system cannot find the path specified: "'/content/gdrive/My Drive/NLP_Capstone_Project/'"
C:\Users\reach\OneDrive\Great Learning\rasa2\Fasttext


In [30]:
!wc fasttext_industrial_Accident.txt

   425  16137 131453 fasttext_industrial_Accident.txt


In [32]:
!head -n 340 fasttext_industrial_Accident.txt > ft_ind_Accident.train
!tail -n 85 fasttext_industrial_Accident.txt > ft_ind_Accident.valid

In [33]:
#fasttext supervised -input fasttext_industrial_data.train -output model_ind_preventive
#fasttext supervised -input ft_ind_data.train -output ft_model

ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train")


In [34]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train")

In [35]:
ft_model_Potential.save_model("ft_model_Potential.bin")

In [36]:
ft_model_Accident.save_model("ft_model_Accident.bin")

In [37]:
ft_model_Potential.predict("accident")

(('__label__4',), array([0.16717224]))

In [38]:
ft_model_Accident.predict("accident")

(('__label__1',), array([0.21756352]))

In [39]:
ft_model_Potential.predict("At 14:16, the operator Samuel was about to open the container for its subsequent loading with silver concentrate,")

(('__label__4',), array([0.16678782]))

In [40]:
ft_model_Accident.predict("At 14:16, the operator Samuel was about to open the container for its subsequent loading with silver concentrate,")

(('__label__1',), array([0.20415291]))

In [41]:
ft_model_Potential.predict("operator of the G-2142 deslaminadora in section 75 was unlocking a sheet of zinc stuck")


(('__label__4',), array([0.16677716]))

In [42]:
ft_model_Accident.predict("operator of the G-2142 deslaminadora in section 75 was unlocking a sheet of zinc stuck")

(('__label__1',), array([0.20362221]))

In [43]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.32941176470588235, 0.32941176470588235)

In [44]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7411764705882353, 0.7411764705882353)

In [45]:
#ft_model.test("ft_ind_data.valid", k=5)

In [46]:
ft_model_Potential.predict("operator of the G-2142", k=5)

(('__label__4', '__label__3', '__label__2', '__label__1', '__label__5'),
 array([0.16699705, 0.16694674, 0.1668057 , 0.166495  , 0.16645721]))

In [47]:
ft_model_Accident.predict("operator of the G-2142", k=5)

(('__label__1', '__label__2', '__label__3', '__label__4', '__label__5'),
 array([0.2124192 , 0.19779292, 0.19710954, 0.196861  , 0.1958673 ]))

In [48]:
#type fasttext_industrial_Potential.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > fasttext_ind_prepro_Potential.txt

In [49]:
!head -n 340 fasttext_industrial_Potential.txt > ft_ind_Potential.train
!tail -n 85 fasttext_industrial_Potential.txt > ft_ind_Potential.valid

In [50]:
!head -n 340 fasttext_industrial_Accident.txt > ft_ind_Accident.train
!tail -n 85 fasttext_industrial_Accident.txt > ft_ind_Accident.valid

In [51]:
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train")
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train")


In [52]:
ft_model_Potential.test("ft_ind_Potential.valid")


(85, 0.32941176470588235, 0.32941176470588235)

In [53]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7411764705882353, 0.7411764705882353)

## **With Hyper parameters tuning with below code such as EPOC,Learnign Rate,Word_grams, hierarchical softmax and Multi label(just tried)**

In [54]:
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.9)

In [55]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.9)

In [56]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.29411764705882354, 0.29411764705882354)

In [57]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7411764705882353, 0.7411764705882353)

In [58]:
#With both Lr and EPOC combined
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.9, epoch=25)

In [59]:
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.9, epoch=25)

In [60]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.4117647058823529, 0.4117647058823529)

In [61]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7176470588235294, 0.7176470588235294)

In [62]:
#With WordNGrams
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.8, epoch=25, wordNgrams=2)
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.8, epoch=25, wordNgrams=2)

In [63]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.3764705882352941, 0.3764705882352941)

In [64]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7176470588235294, 0.7176470588235294)

In [65]:
# With hierarchical softmax
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.7, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.7, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')


In [66]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.2823529411764706, 0.2823529411764706)

In [67]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7294117647058823, 0.7294117647058823)

In [68]:
# Juat tried with Multi labels too
ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
ft_model_Accident = fasttext.train_supervised(input="ft_ind_Accident.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')


In [69]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.35294117647058826, 0.35294117647058826)

In [70]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7411764705882353, 0.7411764705882353)

In [71]:
ft_model_Potential.predict("When using the griff wrench to unscrew the rod from the probe the key came to move by pressing the employee's leg against the probe. ", k=-1, threshold=0.5)

(('__label__3',), array([0.59267658]))

In [72]:
ft_model_Accident.predict("When using the griff wrench to unscrew the rod from the probe", k=-1, threshold=0.5)

(('__label__1',), array([0.78794122]))

In [73]:
ft_model_Potential.test("ft_ind_Potential.valid")

(85, 0.35294117647058826, 0.35294117647058826)

In [74]:
ft_model_Accident.test("ft_ind_Accident.valid")

(85, 0.7411764705882353, 0.7411764705882353)

In [75]:
description = ft_model_Accident.predict("When using the griff wrench to unscrew the rod from the probe", k=-1, threshold=0.5)

In [84]:
print (description[0])

('__label__1',)


In [85]:
print('description')

description


In [ ]:
from flask import Flask,request,jsonify,render_template



app = Flask(__name__) # ,static_url_path='/'

#@app.route('/hello')
#def index():
#    return render_template('index.html',name='John')  #{'hello': 'world'} 


@app.route('/get_p_acc_lvl', methods=['GET'])
def get_potential_accident_level():
    
    query_parameters = request.args
    
    acc_desc = query_parameters.get('description')
    
    print (pred1)
    pred1 = ft_model_Accident.predict(acc_desc, k=-1, threshold=0.5)
    
    print (pred1[0])
    prediction = pred1[0] #ft.predict('acc_desc')
    
    return jsonify(prediction)




if __name__ == '__main__':
  
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
